## Run and Validate ELT Data Pipeline using Dataproc

Let us go ahead and run the ELT Data Pipeline using Dataproce Workflow Template and also validate to see if the Pipeline is successfully run as per the requirements. We will clean up all the workflow runs before taking care of the run and validation of the ELT Data Pipeline.
* Step 1: Pre-run Validation
* Step 2: Run ELT Data Pipeline using Dataproc Workflow Template
* Step 3: Post-run Validation

In [1]:
!gcloud dataproc 

ERROR: (gcloud.dataproc) Command name argument expected.

Available groups for gcloud dataproc:

      autoscaling-policies    Create and manage Dataproc autoscaling policies.
      batches                 Submit Dataproc batch jobs.
      clusters                Create and manage Dataproc clusters.
      jobs                    Submit and manage Dataproc jobs.
      operations              View and manage Dataproc operations.
      workflow-templates      Create and manage Dataproc workflow templates.

For detailed information on this command and its flags, run:
  gcloud dataproc --help


In [2]:
!gcloud dataproc operations

ERROR: (gcloud.dataproc.operations) Command name argument expected.

Available commands for gcloud dataproc operations:

      cancel                  Cancel an active operation.
      delete                  Delete the record of an inactive operation.
      describe                View the details of an operation.
      get-iam-policy          Get IAM policy for an operation.
      list                    View the list of all operations.
      set-iam-policy          Set IAM policy for an operation.

For detailed information on this command and its flags, run:
  gcloud dataproc operations --help


In [3]:
!gcloud dataproc operations list

NAME                                  TIMESTAMP                    TYPE      STATE  ERROR  WARNINGS
b1724f85-0513-42f7-b98a-8afff9262f98  2023-06-12T23:03:42.065448Z  START     DONE
83bd5bd6-9b96-40b0-b524-51200b7c6d50  2023-06-12T01:08:12.761540Z  STOP      DONE
570eca31-a7b2-4614-969e-98dfa71edb41  2023-06-12T00:28:13.973864Z  WORKFLOW  DONE
a280a791-f0ad-44c8-b3da-4a00b82a2686  2023-06-11T23:07:49.191932Z  WORKFLOW  DONE
9f842dee-1f9f-4ba0-bbf7-ddaee65f132d  2023-06-11T22:52:19.754183Z  START     DONE
bc6f0d6b-9d66-37bc-a6f5-0903d0d3fd2b  2023-06-11T01:12:50.149509Z  STOP      DONE
79e2f790-e5ab-4a67-8c4d-c24dc7c40bbf  2023-06-10T23:28:07.504471Z  START     DONE
27b858f2-49d1-4eb5-8727-f5ab0e2d72bb  2023-06-05T01:21:00.601008Z  STOP      DONE
393741d3-ac47-4135-9eed-37bcbe0b853e  2023-06-04T22:37:12.885805Z  START     DONE
e71b50aa-32a9-4739-9b6d-552653917f16  2023-06-04T16:25:22.378387Z  STOP      DONE
4f8576e4-6113-44be-a662-0caec1920510  2023-06-04T15:07:47.701896Z  START     DON

In [ ]:
!gcloud dataproc operations describe d8fdbb7e-dcc5-32c2-bf70-ecd9f3c45ce6

In [ ]:
!gcloud dataproc operations delete 812c4b24-b5e1-4657-a4a8-2783d5c4ed25 --quiet

In [4]:
!gcloud dataproc jobs submit spark-sql --cluster='cluster-256f' -f gs://airetail_mld/scripts/daily_product_revenue/cleanup.sql

Job [b738d3fcfc3748a98c873cafa26188c9] submitted.
Waiting for job output...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/06/12 23:15:19 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/06/12 23:15:19 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/06/12 23:15:19 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/06/12 23:15:19 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
Spark master: yarn, Application Id: application_1686611111024_0001
Time taken: 6.489 seconds
Time taken: 0.74 seconds
23/06/12 23:15:36 WARN io.netty.channel.nio.NioEventLoop: Selector.select() returned prematurely 512 times in a row; rebuilding Selector io.netty.channel.nio.SelectedSelectionKeySetSelector@149a9b67.
Job [b738d3fcfc3748a98c873cafa

In [5]:
!gcloud dataproc workflow-templates describe wf-daily-product-revenue

createTime: '2023-06-12T00:08:35.526710Z'
id: wf-daily-product-revenue
jobs:
- sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/cleanup.sql
  stepId: job-cleanup
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
      table_name: order_items
  stepId: job-convert-order-items
- prerequisiteStepIds:
  - job-cleanup
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/file_format_converter.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
      table_name: orders
  stepId: job-convert-orders
- prerequisiteStepIds:
  - job-convert-orders
  - job-convert-order-items
  sparkSqlJob:
    queryFileUri: gs://airetail_mld/scripts/daily_product_revenue/compute_daily_product_revenue.sql
    scriptVariables:
      bucket_name: gs://airetail_mld
  stepId: job-daily-product-revenue

In [6]:
!gcloud dataproc workflow-templates instantiate wf-daily-product-revenue

Waiting on operation [projects/dataanalytics-347914/regions/us-central1/operations/cb1801b2-eac8-30f4-abdb-7c9d6817455b].
WorkflowTemplate [wf-daily-product-revenue] RUNNING
Job ID job-cleanup-pieh4n6ulpbmg RUNNING
Job ID job-cleanup-pieh4n6ulpbmg COMPLETED
Job ID job-convert-order-items-pieh4n6ulpbmg RUNNING
Job ID job-convert-orders-pieh4n6ulpbmg RUNNING
Job ID job-convert-order-items-pieh4n6ulpbmg COMPLETED
Job ID job-convert-orders-pieh4n6ulpbmg COMPLETED
Job ID job-daily-product-revenue-pieh4n6ulpbmg RUNNING
WorkflowTemplate [wf-daily-product-revenue] DONE
Job ID job-daily-product-revenue-pieh4n6ulpbmg COMPLETED


Validation step

In [1]:
!gcloud dataproc jobs submit spark-sql --cluster='cluster-256f' -e "SHOW databases;"

Job [a8ed2a0584094ce2a9b9982de6a8ac21] submitted.
Waiting for job output...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/06/12 23:24:36 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/06/12 23:24:36 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/06/12 23:24:36 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/06/12 23:24:36 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
Spark master: yarn, Application Id: application_1686611111024_0006
default
retail_bronze_db
retail_gold_db
Time taken: 3.275 seconds, Fetched 3 row(s)
Job [a8ed2a0584094ce2a9b9982de6a8ac21] finished successfully.
done: true
driverControlFilesUri: gs://dataproc-staging-us-central1-936546808722-ripwij5i/google-cloud-dataproc-metainfo/b012612f-63aa-

In [2]:
!gcloud dataproc jobs submit spark-sql --cluster='cluster-256f' -e "USE retail_bronze_db; SHOW tables;"

Job [a7c2f9d297b94233af039915e0fefb3a] submitted.
Waiting for job output...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/06/12 23:25:22 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/06/12 23:25:22 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/06/12 23:25:22 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/06/12 23:25:22 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
Spark master: yarn, Application Id: application_1686611111024_0007
Time taken: 2.198 seconds
retail_bronze_db	order_items	false
retail_bronze_db	orders	false
Time taken: 0.399 seconds, Fetched 2 row(s)
Job [a7c2f9d297b94233af039915e0fefb3a] finished successfully.
done: true
driverControlFilesUri: gs://dataproc-staging-us-central1-936546808722-rip

In [3]:
!gcloud dataproc jobs submit spark-sql --cluster='cluster-256f' -e "USE retail_bronze_db; SELECT COUNT(*) FROM orders;"

Job [baddd9f56cfc4f28878bdeace3298440] submitted.
Waiting for job output...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/06/12 23:25:55 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/06/12 23:25:55 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/06/12 23:25:55 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/06/12 23:25:55 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
Spark master: yarn, Application Id: application_1686611111024_0008
Time taken: 3.119 seconds
68883
Time taken: 7.2 seconds, Fetched 1 row(s)
Job [baddd9f56cfc4f28878bdeace3298440] finished successfully.
done: true
driverControlFilesUri: gs://dataproc-staging-us-central1-936546808722-ripwij5i/google-cloud-dataproc-metainfo/b012612f-63aa-49d4-b6f7-

In [4]:
!gcloud dataproc jobs submit spark-sql --cluster='cluster-256f' -e "USE retail_gold_db; SHOW tables;"

Job [6367ec69a9a34684b368025570330d1f] submitted.
Waiting for job output...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/06/12 23:26:28 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/06/12 23:26:28 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/06/12 23:26:28 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/06/12 23:26:28 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
Spark master: yarn, Application Id: application_1686611111024_0009
Time taken: 2.254 seconds
retail_gold_db	daily_product_revenue	false
Time taken: 0.511 seconds, Fetched 1 row(s)
Job [6367ec69a9a34684b368025570330d1f] finished successfully.
done: true
driverControlFilesUri: gs://dataproc-staging-us-central1-936546808722-ripwij5i/google-cloud-dat

In [5]:
!gcloud dataproc jobs submit spark-sql --cluster='cluster-256f' -e "USE retail_gold_db; SELECT * FROM daily_product_revenue LIMIT 10;"

Job [90563088405c4ceeb1b2733b1ef09842] submitted.
Waiting for job output...
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
23/06/12 23:26:58 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
23/06/12 23:26:58 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
23/06/12 23:26:58 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
23/06/12 23:26:58 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator
Spark master: yarn, Application Id: application_1686611111024_0010
Time taken: 2.149 seconds
2013-07-29 00:00:00.0	957	PENDING_PAYMENT	21	6299.58
2013-08-02 00:00:00.0	1073	COMPLETE	13	2599.87
2013-08-02 00:00:00.0	885	CLOSED	7	174.93
2013-08-02 00:00:00.0	282	SUSPECTED_FRAUD	5	159.95
2013-08-04 00:00:00.0	1004	CLOSED	3	1199.94
2013-08-06 00:00:0